In [2]:
import pandas as pd
import numpy as np
import re,os
import datetime
import json

In [3]:
origin= '/pylon5/sy5fp1p/ehanna/logs/hosts/'

In [54]:
pattern = re.compile('(\d+-\d+-\d+)T(\d+:\d+:\d+)-(\d+:\d+)\s(.*?)\.pvt')

In [63]:
def makenfsMatrix(dateRange):
    startDate = datetime.datetime.strptime(dateRange.split(':')[0],"%Y-%m-%d")
    endDate = datetime.datetime.strptime(dateRange.split(':')[1],"%Y-%m-%d")
    currentDate = startDate
    while (currentDate<endDate):
        nfsTimeouts=[]
        startTime = datetime.datetime.now()
        uniqueTimestamps = []
        allNodes = []
        count=0
        for direc in os.listdir(origin):
            if direc.startswith('r') and direc.endswith('.pvt.bridges.psc.edu'):
                fileName = currentDate.strftime("%Y-%m-%d")+'-'+direc+'.log'
                if fileName in os.listdir(origin+direc+'/'):
                    print(fileName)
                    with open(origin+direc+'/'+fileName) as file:
                        jobRunningCheck = False
                        runningJobsCount=0
                        activeJobNFSErrors = 0
                        nfsErrorCount=0
                        try:
                            for line in file:
                                if 'JID' in line and "prolog started" in line:
                                    runningJobsCount+=1
                                    jobRunningCheck = True
                                    activeJobNFSErrors = 0
                                if jobRunningCheck==False and 'JID' in line and 'epilog ended' in line:
                                    for i in range(activeJobNFSErrors,0,-1):
                                        elems = nfsTimeouts[len(nfsTimeouts)-i].split('|')
                                        nfsTimeouts[len(nfsTimeouts)-i] = elems[0]+'|'+elems[1]+'|'+elems[2]+'|'+'True'
                                if jobRunningCheck and 'JID' in line and 'epilog ended' in line:
                                    if runningJobsCount==1:
                                        jobRunningCheck = False                                            
                                    runningJobsCount-= 1
                                if 'not responding' in line and 'nfs: server ' in line:
                                    activeJobNFSErrors +=1
                                    match = re.search(pattern,line)
                                    if match is not None:
                                        date = match.group(1)
                                        time = match.group(2)
                                        Zone = match.group(3)
                                        node = match.group(4)
                                        modTime = date+'|'+time
                                        if jobRunningCheck:
                                            nfsTimeouts.append(modTime+'|'+node+'|'+'True')
                                        else:
                                            nfsTimeouts.append(modTime+'|'+node+'|'+'False')
                        except Exception as e:
                            print(e)
        text = ''
        for i in nfsTimeouts:
            text += i+'\n'
        file = open("NFS-Error-Text"+currentDate.strftime("%Y-%m-%d")+".txt","w")
        file.write(text)
        file.close()
        currentDate = currentDate+datetime.timedelta(days=1)

In [ ]:
makenfsMatrix('2018-01-22:2018-01-23')

In [12]:
# curr_dir = './'
# allColumns  = ['Date','Time']
# dictStringNodes = ''
# withinJobDict = {}
# outOfJobDict = {}
# for i in range(1,753):
#     allColumns.append('r'+str(i).zfill(3))
#     withinJobDict['r'+str(i).zfill(3)]=0
#     outOfJobDict['r'+str(i).zfill(3)]=0
# withinJobdf = pd.DataFrame(columns=allColumns)
# outOfJobdf = pd.DataFrame(columns=allColumns)
# count=0
# for direc in os.listdir(curr_dir):
#     if direc.startswith("NFS-Error-Text"):
#         with open(curr_dir+direc) as file:
#             print(direc)
#             for line in file:
#                 count+=1
#                 print(count)
#                 elems = line.split('|')
#                 date = elems[0]
#                 time = elems[1]
#                 time = time.split(':')[0]+':'+time.split(':')[1]+':'+'00'
#                 node = elems[2]
#                 jobStatus = elems[3]
#                 if jobStatus:
#                     if date in withinJobdf['Date'] and time in withinJobdf['Time']:
#                         indexes = withinJobdf[(withinJobdf['Date']==date)&(withinJobdf['Time']==time)].index
#                         withinJobdf.loc[indexes,node]+=1
#                     else:
#                         tempDF = pd.DataFrame(0,index=np.arange(1), columns=allColumns)
#                         tempDF['Date'] = date
#                         tempDF['Time'] = time
#                         tempDF.iloc[:,2:] = 0
#                         tempDF[node]+=1
#                         withinJobdf.append(tempDF)
#                 else:
#                     if date in outOfJobdf['Date'] and time in outOfJobdf['Time']:
#                         indexes = outOfJobdf[(outOfJobdf['Date']==date)&(outOfJobdf['Time']==time)].index
#                         outOfJobdf.loc[indexes,node]+=1
#                     else:
#                         tempDF = pd.DataFrame(0,index=np.arange(1),columns=allColumns)
#                         tempDF['Date'] = date
#                         tempDF['Time'] = time
#                         tempDF.iloc[:,2:] = 0
#                         tempDF[node]+=1
#                         outOfJobdf.append(tempDF)
# withinJobdf.to_csv("WithinJob.csv",index=False)
# outOfJobdf.to_csv("OutOfJob.csv",index=False)

NFS-Error-Text2018-01-22.txt
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270

KeyboardInterrupt: 

In [8]:
curr_dir = './'
allColumns  = ['Date','Time']
rNodesDict = {}
withinJobDict = {}
outOfJobDict = {}
for i in range(1,753):
    allColumns.append('r'+str(i).zfill(3))
    rNodesDict['r'+str(i).zfill(3)]=0
withinJobdf = pd.DataFrame(columns=allColumns)
outOfJobdf = pd.DataFrame(columns=allColumns)
for direc in os.listdir(curr_dir):
    if direc.startswith("NFS-Error-Text"):
        with open(curr_dir+direc) as file:
            for line in file:
                line = line.strip('\n')
                elems = line.split('|')
                date = elems[0]
                time = elems[1]
                time = time.split(':')[0]+':'+time.split(':')[1]+':'+'00'
                dateTime = date+'_'+time
                node = elems[2]
                jobStatus = elems[3]
                if jobStatus=='True':
                    if dateTime in withinJobDict.keys():
                        withinJobDict[dateTime][node]+=1
                    else:
                        withinJobDict[dateTime]=rNodesDict.copy()
                        withinJobDict[dateTime][node]+=1
                else:
                    if dateTime in outOfJobDict.keys():
                        outOfJobDict[dateTime][node]+=1
                    else:
                        outOfJobDict[dateTime]=rNodesDict.copy()
                        outOfJobDict[dateTime][node]+=1
withinJobJson = json.dumps(withinJobDict)
withinJobDf = pd.read_json(withinJobJson,orient='index')
withinJobDf.to_csv("WithinJob.csv",index=True)
outOfJobJson = json.dumps(outOfJobDict)
outOfJobDf = pd.read_json(outOfJobJson,orient='index')
outOfJobDf.to_csv("outOfJob.csv",index=True)